In [5]:
import pandas as pd
import requests
from datetime import datetime
import os


In [6]:
def remove_multiindex(df:pd.DataFrame):
	return df.droplevel(level=0,axis=1)

In [7]:
# link for extract html data
def getdata(url):
    r = requests.get(url)
    return r.text

def get_nba_season_year(date=None):
    if date is None:
        date = datetime.now()
    return date.year + 1 if date.month >= 10 else date.year

In [8]:
full_df_list = pd.DataFrame(columns=[ 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
		   '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
		   'BLK', 'TOV', 'PF', 'PTS', 'Age', 'W', 'L', 'MOV',
		   'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%',])


for year in range(1980,get_nba_season_year() + 1):
	print('Fetching',year)

	if os.path.exists(f'team_stats/{year}_tm_stats.csv'):
		new = pd.read_csv(f'team_stats/{year}_tm_stats.csv')
	else:

		htmldata = getdata(f"https://www.basketball-reference.com/leagues/NBA_{year}.html")
		df_list = pd.read_html(htmldata)

		if year <= 2015:
			per_game_stat = df_list[3].copy()

			adv_stat = df_list[8].copy()
		else:
			per_game_stat = df_list[4].copy()

			adv_stat = df_list[10].copy()

		new = pd.merge(per_game_stat,adv_stat.droplevel(level=0,axis=1),on='Team')

		new = new[[ 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
		   '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
		   'BLK', 'TOV', 'PF', 'PTS', 'Age', 'W', 'L', 'MOV',
		   'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%',]].sort_values('W',ascending=False).reset_index(drop=True)

		new['Year'] = year

		new.loc[new['Team'] == 'League Average','Team'] = f'League Average {year}'

		new.to_csv(f'team_stats/{year}_tm_stats.csv')
	full_df_list = pd.concat([full_df_list,new])

	print(year,'Complete')


full_df_list.to_csv('team_stats/all.csv')

Fetching 1980
1980 Complete
Fetching 1981
1981 Complete
Fetching 1982
1982 Complete
Fetching 1983
1983 Complete
Fetching 1984
1984 Complete
Fetching 1985
1985 Complete
Fetching 1986
1986 Complete
Fetching 1987
1987 Complete
Fetching 1988
1988 Complete
Fetching 1989
1989 Complete
Fetching 1990
1990 Complete
Fetching 1991
1991 Complete
Fetching 1992
1992 Complete
Fetching 1993
1993 Complete
Fetching 1994
1994 Complete
Fetching 1995
1995 Complete
Fetching 1996
1996 Complete
Fetching 1997
1997 Complete
Fetching 1998
1998 Complete
Fetching 1999
1999 Complete
Fetching 2000
2000 Complete
Fetching 2001
2001 Complete
Fetching 2002
2002 Complete
Fetching 2003
2003 Complete
Fetching 2004
2004 Complete
Fetching 2005
2005 Complete
Fetching 2006
2006 Complete
Fetching 2007
2007 Complete
Fetching 2008
2008 Complete
Fetching 2009
2009 Complete
Fetching 2010
2010 Complete
Fetching 2011
2011 Complete
Fetching 2012
2012 Complete
Fetching 2013
2013 Complete
Fetching 2014
2014 Complete
Fetching 2015
2015 C

/var/folders/0p/ttyhqytj0zxf3tq1njp37rqw0000gn/T/ipykernel_88687/375069677.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df_list = pd.concat([full_df_list,new])
/var/folders/0p/ttyhqytj0zxf3tq1njp37rqw0000gn/T/ipykernel_88687/375069677.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(htmldata)


2024 Complete


In [9]:
full_df_list.to_csv('../Data/Team Data/Combined/league_stats.csv')